In [1]:
import arcpy

In [2]:
# Input database
# for future use: edit "ky_small_scale"
inputDB = 'c:\\SchimpffGIS\\data\\ky.gdb'

# Create string representing output locations
myData = 'c:\\SchimpffGIS\\data'
myDB = 'landcover.gdb'
outputDB = f'{myData}\\{myDB}'

# Verify
print(f'Input: {inputDB} Output: {outputDB}')

# AOI in input database
# for future use: edit 'state_polygon'
aoi = 'state_polygon'

Input: c:\SchimpffGIS\data\ky.gdb Output: c:\SchimpffGIS\data\landcover.gdb


In [3]:
# Input database
# for future use: edit env.workspace location
arcpy.env.workspace = 'c:\\SchimpffGIS\\data\\ky.gdb'
# Define output CRS
ky = arcpy.SpatialReference(3089)
# Set the output CRS
arcpy.env.outputCoordinateSystem = ky
# CRS is WGS84
wgs84 = arcpy.SpatialReference(4326)
# Enable overwriting exist layers
arcpy.env.overwriteOutput = True

<Result 'c:\\SchimpffGIS\\data\\ky_small_scale.gdb\\aoi'>

<Result 'c:\\SchimpffGIS\\data\\ky_small_scale.gdb\\aoi_polygon'>

In [8]:
print(outputDB)

c:\SchimpffGIS\data\landcover.gdb


In [9]:
# Creating output database
print("Creating workspace geodatabase...")
# Check if it already exists
if arcpy.Exists(outputDB):
    print(f"Output database exists: {outputDB}")
else:
    arcpy.management.CreateFileGDB(myData, myDB)
    print(f"Created output database: {outputDB}")
    
print(f"Using {arcpy.env.workspace} for input data.")

Creating workspace geodatabase...
Output database exists: c:\SchimpffGIS\data\landcover.gdb
Using c:\SchimpffGIS\data\ky.gdb for input data.


In [16]:
aoi_polygon = "state_polygon"
print(aoi_polygon)
features = arcpy.ListFeatureClasses()
print(features)

state_polygon
['counties', 'gnis', 'incoporated_places', 'state_polygon', 'roads', 'streams', 'waterbodies', 'miguels', 'one_mi_miguels', 'gnis_miguels', 'KingdomComeStatePark', 'fiveMiKingdom', 'gnisKCSP']


In [17]:
# Clip vector layers
# Create a list of feature class names
features = arcpy.ListFeatureClasses()
# Loop over list of features
for feature in features:
    # Make output name
    output = f"{outputDB}\\{feature}"
    # If aoi, then don't clip; copy
    if feature == aoi_polygon: #aoi_polygon
        arcpy.management.Copy(feature, output)
        print(f"Copying {feature} to {output}")
    else:
        print(f"Clipping {feature} with {aoi} and outputting layer to {output} ...")
        arcpy.analysis.Clip(feature, aoi_polygon, output)

# Clip raster layers
# Create a list of raster names
rasters = arcpy.ListRasters()
# Loop over list of rasters
for raster in rasters:
    output = f"{outputDB}\\{raster}"
    print(f"Clipping {raster} with {aoi} and outputting layer to {output} ...")
    arcpy.management.Clip(raster, "#", output, aoi_polygon, "#", "ClippingGeometry", "#")

Clipping counties with state_polygon and outputting layer to c:\SchimpffGIS\data\landcover.gdb\counties ...
Clipping gnis with state_polygon and outputting layer to c:\SchimpffGIS\data\landcover.gdb\gnis ...
Clipping incoporated_places with state_polygon and outputting layer to c:\SchimpffGIS\data\landcover.gdb\incoporated_places ...
Copying state_polygon to c:\SchimpffGIS\data\landcover.gdb\state_polygon
Clipping roads with state_polygon and outputting layer to c:\SchimpffGIS\data\landcover.gdb\roads ...
Clipping streams with state_polygon and outputting layer to c:\SchimpffGIS\data\landcover.gdb\streams ...
Clipping waterbodies with state_polygon and outputting layer to c:\SchimpffGIS\data\landcover.gdb\waterbodies ...
Clipping miguels with state_polygon and outputting layer to c:\SchimpffGIS\data\landcover.gdb\miguels ...
Clipping one_mi_miguels with state_polygon and outputting layer to c:\SchimpffGIS\data\landcover.gdb\one_mi_miguels ...
Clipping gnis_miguels with state_polygon an

In [18]:
arcpy.env.workspace = outputDB

In [19]:
print(arcpy.env.workspace)

c:\SchimpffGIS\data\landcover.gdb


In [20]:
sumCover = 0
with arcpy.da.SearchCursor('NLCD_2016_land_cover', ['Count']) as cursor:
    for row in cursor:
        sumCover += row[0]
with arcpy.da.SearchCursor('NLCD_2016_land_cover', ['Count', 'NLCD_Land_Cover_Class']) as cursor:
    for row in cursor:
        a = (row[0]/sumCover) * 100
        b = (row[0] * 98.4**2) / 2.788e+7
        a = round(a, 2)
        b = round(b, 1)
        c = f'{a}%, {b} sq mi of {row[1]}'
        print(c)

1.86%, 749.6 sq mi of Open Water
4.57%, 1847.1 sq mi of Developed, Open Space
1.6%, 646.1 sq mi of Developed, Low Intensity
0.69%, 278.7 sq mi of Developed, Medium Intensity
0.28%, 112.3 sq mi of Developed, High Intensity
0.4%, 162.4 sq mi of Barren Land
42.9%, 17326.0 sq mi of Deciduous Forest
1.03%, 417.3 sq mi of Evergreen Forest
9.76%, 3941.0 sq mi of Mixed Forest
0.73%, 294.0 sq mi of Shrub/Scrub
1.29%, 521.2 sq mi of Herbaceuous
21.83%, 8814.0 sq mi of Hay/Pasture
11.75%, 4744.7 sq mi of Cultivated Crops
1.12%, 453.6 sq mi of Woody Wetlands
0.19%, 75.0 sq mi of Emergent Herbaceuous Wetlands
